In [13]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Load .env file
load_dotenv()

# Get values from .env
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

# Build PostgreSQL connection URL
connection_string = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_string)

# Query your table
query = "SELECT * FROM software_salaries"  # Replace with your actual table name
df = pd.read_sql(query, engine)

# Display sample
print(df.head())


            job_title experience_level employment_type company_size  \
0        Data Analyst              Mid        Contract       Medium   
1     DevOps Engineer              Mid        Contract        Small   
2  Research Scientist             Lead            None       Medium   
3       Software Engr             Lead       Full-time        Large   
4       Software Engr             Lead          Intern        Large   

  company_location  remote_ratio salary_currency  years_experience  \
0          Germany             0             INR                13   
1            India           100             GBP                 9   
2          Germany             0             EUR                19   
3            India            50             INR                 7   
4          Germany           100             INR                10   

     base_salary  bonus  stock_options   total_salary  salary_in_usd currency  \
0   68407.451747   1100          19325   88832.451747    1065.989421   

In [14]:
# Shape and types
print(df.shape)
print(df.info())

# Summary stats
print(df.describe(include='all'))

# Nulls and duplicates
print(df.isnull().sum())
print(df.duplicated().sum())

(100000, 18)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   job_title           100000 non-null  object 
 1   experience_level    80000 non-null   object 
 2   employment_type     76016 non-null   object 
 3   company_size        100000 non-null  object 
 4   company_location    100000 non-null  object 
 5   remote_ratio        100000 non-null  int64  
 6   salary_currency     100000 non-null  object 
 7   years_experience    100000 non-null  int64  
 8   base_salary         100000 non-null  float64
 9   bonus               100000 non-null  int64  
 10  stock_options       100000 non-null  int64  
 11  total_salary        100000 non-null  float64
 12  salary_in_usd       100000 non-null  float64
 13  currency            100000 non-null  object 
 14  education           0 non-null       object 
 15  skills              0 

In [15]:
df.drop(columns=['education', 'skills'], inplace=True)


In [16]:
df = df[(df['base_salary'] >= 0) & (df['bonus'] >= 0) & (df['stock_options'] >= 0) & (df['adjusted_total_usd'] >= 0)]


In [17]:
df.drop_duplicates(inplace=True)


In [18]:
df['experience_level'].fillna(df['experience_level'].mode()[0], inplace=True)
df['employment_type'].fillna(df['employment_type'].mode()[0], inplace=True)


C:\Users\Minfy\AppData\Local\Temp\ipykernel_14836\2714007931.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['experience_level'].fillna(df['experience_level'].mode()[0], inplace=True)
C:\Users\Minfy\AppData\Local\Temp\ipykernel_14836\2714007931.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

In [19]:
df['is_remote'] = df['remote_ratio'].apply(lambda x: 1 if x == 100 else 0)
df['total_salary'] = df['base_salary'] + df['bonus'] + df['stock_options']

In [28]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
categorical_cols = ['job_title', 'experience_level', 'employment_type', 'company_size', 'company_location', 'salary_currency']

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Store encoder if you need to inverse_transform later
df['currency'] = le.fit_transform(df['salary_currency'])  # now it's numeric



In [21]:
scaler = MinMaxScaler()
numeric_cols = ['years_experience', 'base_salary', 'bonus', 'stock_options', 'conversion_rate']
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [22]:
import numpy as np
df['adjusted_total_usd_log'] = np.log1p(df['adjusted_total_usd'])


In [23]:
X = df.drop(columns=['adjusted_total_usd', 'adjusted_total_usd_log'])  # or drop other unused
y = df['adjusted_total_usd_log']  # log target

In [30]:
print(X.dtypes)  # All should be int or float


job_title             int32
experience_level      int32
employment_type       int32
company_size          int32
company_location      int32
remote_ratio          int64
salary_currency       int32
years_experience    float64
base_salary         float64
bonus               float64
stock_options       float64
total_salary        float64
salary_in_usd       float64
currency             object
conversion_rate     float64
is_remote             int64
dtype: object


In [29]:
import shap
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# 9. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 10. Try multiple models
models = {
    'XGBoost': xgb.XGBRegressor(objective='reg:squarederror', random_state=42),
    'RandomForest': RandomForestRegressor(random_state=42),
    'LinearRegression': LinearRegression()
}

model_scores = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred_log = model.predict(X_test)
    y_pred = np.expm1(y_pred_log)
    y_true = np.expm1(y_test)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    model_scores[name] = {'mae': mae, 'r2': r2}
    print(f"{name} - MAE: {mae:.2f}, R2: {r2:.4f}")

# 11. Select the best model
best_model_name = max(model_scores, key=lambda x: model_scores[x]['r2'])
best_model = models[best_model_name]
print(f"Best Model: {best_model_name}")

# 12. SHAP Explainability (only for tree models)
if best_model_name in ['XGBoost', 'RandomForest']:
    explainer = shap.Explainer(best_model)
    shap_values = explainer(X_test)
    shap.plots.beeswarm(shap_values)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:currency: object